# **Website Log Data Cleaning and Sessionization**

## **Overview**
This notebook demonstrates the process of cleaning raw Apache web server access
logs and transforming them into meaningful user sessions. The goal is to
identify genuine user behavior by grouping requests into sessions using a
30-minute inactivity threshold and removing bot/crawler traffic.

This type of analysis is commonly used in web analytics to understand
user engagement and navigation patterns.


In [40]:
import pandas as pd
import re

## **Dataset Description**
The dataset used in this project is a public Apache web server access log
containing real website traffic. Each log entry represents a single HTTP request
and includes information such as the visitor’s IP address, request timestamp,
requested URL, and user-agent string.

The dataset contains a mix of human users and automated bots, making it suitable
for sessionization and bot filtering tasks.


In [43]:
log_path = "../data/access.txt"
logs = []

## **Parsing Raw Log Data**
Apache access logs are stored as unstructured text. To enable analysis, the raw
log entries are parsed using regular expressions and converted into a structured
tabular format with clearly defined columns such as IP address, timestamp,
request method, URL, and user-agent.


In [45]:
pattern = re.compile(
    r'(\d+\.\d+\.\d+\.\d+).*?\[(.*?)\]\s+"(\w+)\s+(.*?)\s+HTTP.*?"\s+\d+\s+\S+\s+".*?"\s+"(.*?)"'
)

with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
    for line in f:
        match = pattern.search(line)
        if match:
            logs.append(match.groups())


In [47]:
df = pd.DataFrame(
    logs,
    columns=["ip_address", "timestamp", "method", "url", "user_agent"]
)

df.head()


,ip_address,timestamp,method,url,user_agent
0,83.149.9.216,17/May/2015:10:05:03 +0000,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
1,83.149.9.216,17/May/2015:10:05:43 +0000,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
2,83.149.9.216,17/May/2015:10:05:47 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
3,83.149.9.216,17/May/2015:10:05:12 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
4,83.149.9.216,17/May/2015:10:05:07 +0000,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...


## **Timestamp Processing**
The timestamp field is converted into a datetime format to enable time-based
analysis. This step is essential for calculating time differences between
requests and identifying user sessions.

#### 1. Convert timestamp to datetime

In [4]:
df["timestamp"] = pd.to_datetime(
    df["timestamp"],
    format="%d/%b/%Y:%H:%M:%S %z"
)

df.head()

,ip_address,timestamp,method,url,user_agent
0,83.149.9.216,2015-05-17 10:05:03+00:00,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
1,83.149.9.216,2015-05-17 10:05:43+00:00,GET,/presentations/logstash-monitorama-2013/images...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
2,83.149.9.216,2015-05-17 10:05:47+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
3,83.149.9.216,2015-05-17 10:05:12+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...
4,83.149.9.216,2015-05-17 10:05:07+00:00,GET,/presentations/logstash-monitorama-2013/plugin...,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_1)...


#### 2. Sort data by user and time

In [7]:
df = df.sort_values(by=["ip_address", "timestamp"])
df.head()

,ip_address,timestamp,method,url,user_agent
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...


#### 3. Calculate time difference between requests

In [10]:
df["time_diff"] = df.groupby("ip_address")["timestamp"].diff()
df.head()

,ip_address,timestamp,method,url,user_agent,time_diff
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03


#### 4. Apply 30-minute session rule

In [13]:
session_timeout = pd.Timedelta(minutes=30)

df["new_session"] = (
    (df["time_diff"].isna()) | (df["time_diff"] > session_timeout)
)
df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False


## **User Session Identification**
User sessions are identified by grouping requests by IP address and ordering them
chronologically. A new session is created whenever the time difference between
two consecutive requests exceeds 30 minutes. This inactivity-based approach is
commonly used in web analytics to model real user browsing behavior.


#### 1. Create session_id

In [16]:
df["session_id"] = (
    df.groupby("ip_address")["new_session"].cumsum()
)

df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1


#### 2. Make session_id more readable

In [19]:
df["session_id"] = (
    df["ip_address"] + "_S" + df["session_id"].astype(str)
)

df.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1.22.35.226_S1
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1.22.35.226_S1
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1.22.35.226_S1
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1.22.35.226_S1
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1.22.35.226_S1


## **Bot and Crawler Filtering**
Web traffic often includes automated bots and crawlers that do not represent real
user behavior. These requests are identified using keywords in the user-agent
field (such as 'bot', 'spider', and 'crawler') and removed from the dataset to
ensure accurate session analysis.


#### 1. Identify bot / crawler traffic

In [23]:
bot_keywords = [
    "bot", "spider", "crawl", "slurp", "archiver", "crawler"
]

df["is_bot"] = df["user_agent"].str.lower().str.contains(
    "|".join(bot_keywords),
    na=False
)

df[["user_agent", "is_bot"]].head()

,user_agent,is_bot
5857,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5862,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5858,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5863,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False
5855,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,False


#### 2. Remove bot traffic

In [26]:
df_clean = df[~df["is_bot"]].copy()
df_clean.head()

,ip_address,timestamp,method,url,user_agent,time_diff,new_session,session_id,is_bot
5857,1.22.35.226,2015-05-19 11:05:07+00:00,GET,/style2.css,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,NaT,True,1.22.35.226_S1,False
5862,1.22.35.226,2015-05-19 11:05:27+00:00,GET,/images/jordan-80.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:20,False,1.22.35.226_S1,False
5858,1.22.35.226,2015-05-19 11:05:36+00:00,GET,/favicon.ico,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:09,False,1.22.35.226_S1,False
5863,1.22.35.226,2015-05-19 11:05:43+00:00,GET,/images/web/2009/banner.png,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:07,False,1.22.35.226_S1,False
5855,1.22.35.226,2015-05-19 11:05:46+00:00,GET,/projects/xdotool/,Mozilla/5.0 (Windows NT 6.1; rv:26.0) Gecko/20...,0 days 00:00:03,False,1.22.35.226_S1,False


In [28]:
print("Original rows:", len(df))
print("After bot removal:", len(df_clean))

Original rows: 9999
After bot removal: 8602


## **Session-Level Summary**
After sessionization and bot removal, the data is aggregated at the session level.
The final output summarizes each user session with metrics such as session start
time, session end time, number of page views, and unique pages visited. This
session-level dataset can be directly used for downstream user behavior analysis
and reporting.


#### 1. Create final session-level summary

In [31]:
session_summary = (
    df_clean
    .groupby(["session_id", "ip_address"])
    .agg(
        session_start=("timestamp", "min"),
        session_end=("timestamp", "max"),
        page_views=("url", "count"),
        unique_pages=("url", "nunique")
    )
    .reset_index()
)

session_summary.head()

,session_id,ip_address,session_start,session_end,page_views,unique_pages
0,1.22.35.226_S1,1.22.35.226,2015-05-19 11:05:07+00:00,2015-05-19 11:05:49+00:00,6,6
1,100.2.4.116_S1,100.2.4.116,2015-05-18 21:05:22+00:00,2015-05-18 21:05:53+00:00,2,2
2,100.2.4.116_S2,100.2.4.116,2015-05-18 23:05:16+00:00,2015-05-18 23:05:23+00:00,2,1
3,100.2.4.116_S3,100.2.4.116,2015-05-19 04:05:22+00:00,2015-05-19 04:05:31+00:00,2,2
4,101.119.18.35_S1,101.119.18.35,2015-05-19 16:05:00+00:00,2015-05-19 16:05:56+00:00,33,33


#### 2. Save final output to CSV

In [37]:
session_summary.to_csv(
    "../data/sessionized_logs.csv",
    index=False
)

## **Conclusion**
This analysis demonstrates how raw web server logs can be transformed into
meaningful session-level insights using Python and Pandas. The resulting dataset
provides a clean and structured view of user behavior and can support further
analysis such as funnel analysis, engagement metrics, or conversion tracking.

